In [ ]:
# 线程，直接调用
import threading
import time
def Princ(String):
    print('task', String)
    time.sleep(5)
# target=目标函数， args=传入的参数
t1 = threading.Thread(target=Princ, args=('t1',))
t1.start()
t2 = threading.Thread(target=Princ, args=('t1',))
t2.start()
t3 = threading.Thread(target=Princ, args=('t1',))
t3.start()

In [ ]:
# 线程，类调用
import threading
import time
class MyThreading(threading.Thread):
    def __init__(self, conn):
        super(MyThreading, self).__init__()
        self.conn = conn
    def run(self):
        print('run task', self.conn)
        time.sleep(5)
t1 = MyThreading('t1')
t2 = MyThreading('t2')
t1.start()
t2.start()

In [ ]:
# 线程，子线程并行执行
import threading
import time
def Princ(String):
    print('task', String)
    time.sleep(2)
# 执行子线程的时间
start_time = time.time()
# 存放线程的实例
t_objs = []
for i in range(50):
    t = threading.Thread(target=Princ, args=('t-%s' % (i),))
    t.start()
    # 为了不让后面的子线程阻塞，把当前的子线程放入到一个列表中
    t_objs.append(t)
# 循环所有子线程实例，等待所有子线程执行完毕
for t in t_objs:
    t.join()
# 当前时间减去开始时间就等于执行的过程中需要的时间
print(time.time() - start_time)

In [ ]:
#递归锁
import threading
def run1():
    print("grab the first part data")
    lock.acquire()
    global num
    num += 1
    lock.release()
    return num
def run2():
    print("grab the second part data")
    lock.acquire()
    global num2
    num2 += 1
    lock.release()
    return num2
def run3():
    lock.acquire()
    res = run1()
    print('--------between run1 and run2-----')
    res2 = run2()
    lock.release()
    print(res, res2)
t_objs = []
if __name__ == '__main__':
    num, num2 = 0, 0
    lock = threading.RLock()  # RLock()类似创建了一个字典，每次退出的时候找到字典的值进行退出
    # lock = threading.Lock()  # Lock()会阻塞在这儿
    for i in range(10):
        t = threading.Thread(target=run3)
        t.start()
        t_objs.append(t)
for t in t_objs:
    t.join()
print(num, num2)

In [ ]:
# 信号量(Semaphore)，互斥锁同时只允许一个线程更改数据，而Semaphore是同时允许一定数量的线程更改数据
import threading
import time
def run(n):
    semaphore.acquire()  # 获取信号，信号可以有多把锁
    time.sleep(1)  # 等待一秒钟
    print("run the thread: %s\n" % n)
    semaphore.release()  # 释放信号
t_objs = []
if __name__ == '__main__':
    semaphore = threading.BoundedSemaphore(5)  # 声明一个信号量，最多允许5个线程同时运行
    for i in range(20):  # 运行20个线程
        t = threading.Thread(target=run, args=(i,))  # 创建线程
        t.start()  # 启动线程
        t_objs.append(t)
for t in t_objs:
    t.join()
print('>>>>>>>>>>>>>')

In [ ]:
# 进程内启动线程
from multiprocessing import Process
import threading
def Thread(String):
    print(String)
def Proces(String):
    print('hello', String)
    t = threading.Thread(target=Thread, args=('Thread %s' % (String),))  # 创建一个线程
    t.start()  # 启动它
if __name__ == '__main__':
    p = Process(target=Proces, args=('World',))  # 创建一个进程
    p.start()  # 启动
    p.join()  # 等待进程执行完毕

In [ ]:
# 启动多进程
from multiprocessing import Process
import time
def f(name):
    time.sleep(0.2)
    print('hello', name)
if __name__ == '__main__':
    for n in range(10):  # 创建一个进程
        p = Process(target=f, args=('bob %s' % (n),))
        # 启动
        p.start()
        # 等待进程执行完毕

In [ ]:
#进程Queue
# _*_ coding:utf-8 _*_
from multiprocessing import Process, Queue
def ChildProcess(Q):
    Q.put(['Hello', None, 'World'])  # 在Queue里面上传一个列表
if __name__ == '__main__':
    q = Queue()  # 创建一个Queue
    p = Process(target=ChildProcess, args=(q,))  # 创建一个子进程，并把Queue传给子进程,相当于克隆了一份Queue
    p.start()  # 启动子进程
    print(q.get())  # 获取q中的数据
    p.join()

In [ ]:
#数据共享manager
# _*_ coding:utf-8 _*_
from multiprocessing import Process, Manager
import os

def ChildProcess(Dict, List):
    Dict['k1'] = 'v1'
    Dict['k2'] = 'v2'
    List.append(os.getpid())  # 获取子进程的PID
    print(List)  # 输出列表中的内容
    
if __name__ == '__main__':
    manager = Manager()  # 生成Manager对象
    Dict = manager.dict()  # 生成一个可以在多个进程之间传递共享的字典
    List = manager.list()  # 生成一个字典
    ProcessList = []  # 创建一个空列表，存放进程的对象，等待子进程执行用于
    for i in range(10):  # 生成是个子进程
        p = Process(target=ChildProcess, args=(Dict, List))  # 创建一个子进程
        p.start()  # 启动
        ProcessList.append(p)  # 把子进程添加到p_list列表中
    for res in ProcessList:  # 循环所有的子进程
        res.join()  # 等待执行完毕
    print('\n')
    print(Dict)
    print(List)

In [ ]:
#管道pipe
# _*_ coding:utf-8 _*_
from multiprocessing import Process, Pipe
def ChildProcess(conn):
    conn.send(['Hello', None, 'World'])  # 写一段数据
    conn.close()  # 关闭
if __name__ == '__main__':
    parent_conn, child_conn = Pipe()  # 生成一个管道实例，parent_conn, child_conn管道的两头
    p = Process(target=ChildProcess, args=(child_conn,))
    p.start()
    print(parent_conn.recv())  # 收取消息
    p.join()

In [ ]:
# 线程池
#!/usr/bin/env python
# -*- coding:utf-8 -*-
import threading
import queue
import time
class MyThread:
    def __init__(self,max_num=10):
        self.queue = queue.Queue()
        for n in range(max_num):
            self.queue.put(threading.Thread)
    def get_thread(self):
        return self.queue.get()
    def put_thread(self):
        self.queue.put(threading.Thread)
pool = MyThread(5)
def RunThread(arg,pool):
    print(arg)
    time.sleep(2)
    pool.put_thread()
for n in range(30):
    thread = pool.get_thread()
    t = thread(target=RunThread, args=(n,pool,))
    t.start()